### Preliminary submission looking at basic inter-likeness of clikcing an ad

thought list:
- how many times has an ad appeared?
- how many times has an ad been clicked?
- % of time the ad was clicked vs appeared
- the more an ad appears the higher chance it has of being clicked (as a baseline truth, not thinking about locale, user diversity, etc)
- can I generate a weight to rank each ad based on the above properties?
- if so we can use this to sort the ads and group by display_id
- This could provide a fair baseline based soley on click-by-show popularity not taking into account the other data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('ggplot')
import collections
import json
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
#train = pd.read_csv("data/clicks_train.csv")
temp_train = pd.read_csv("data_mini/clicks_train_mini.csv")

In [ ]:
def df_info(df):
    print 'temp_train: ', df.shape
    print 'Num of unique display_id: ', len(df['display_id'].unique())
    print 'Num of unique ad_id: ', len(df['ad_id'].unique())
    print 'Click ratio: {}% of ads clicked.'.format(round(float(df['clicked'].value_counts()[1])/df.shape[0]*100,3))
    display(df.head())

df_info(temp_train)

In [ ]:
temp_train_copy = temp_train.copy()
# value = temp_train_copy.groupby('ad_id').count()
# value

display(temp_train_copy.loc[temp_train_copy['ad_id'] == 347571].shape[0])
display(temp_train_copy.loc[temp_train_copy['ad_id'] == 347571].loc[temp_train_copy['clicked'] == 1].shape[0])
4.0/12

In [ ]:
# could add here a normalizer value to account for ads with very low and very high occurances.

# VERY SLOW... c'mon we can make this better.... took 20 sec

#gives me a dict where the key is the ad_id and the value is a tuple with information about its click ratio
unique_ads = temp_train['ad_id'].unique().tolist()
ad_count_dict = {}
for ad in unique_ads:
#     temp_tuple = () #3 elements for clicked, not clicked, and %
    
    # KYLE: Next line just keeps the part of the table that we want
    # Maybe it is faster??
    temp = temp_train_copy.loc[temp_train_copy['ad_id'] == ad]
    
    appeared = temp.shape[0]
    clicked = temp.loc[temp_train_copy['clicked'] == 1].shape[0]
    not_clicked = temp.loc[temp_train_copy['clicked'] == 0].shape[0]
    percent_clicked = float(clicked)/appeared
    
    temp_tuple = (percent_clicked, appeared, clicked, not_clicked)
    
    
    ad_count_dict[ad] = temp_tuple
ad_count_dict
    

In [ ]:
#gives me a dict where the key is the display_id and the value is a list of all the ad_ids associated with it

display_list = temp_train['display_id'].unique().tolist()
display_dict = {}
for display in display_list:
    ad_list = temp_train.loc[temp_train['display_id'] == display]['ad_id'].tolist()
    
    display_dict[display] = ad_list
display_dict

In [ ]:
# sorted lists of ad_id based on click probability binded to a key of display_id

display_dict_sorted = {}
for key, value in display_dict.iteritems():
    temp_list = []
    for e in value:
        temp_list.append(ad_count_dict[e][0])
    
    sorted_list = [x for (y,x) in sorted(zip(temp_list, value), reverse=True)]
    sorted_list = map(str, sorted_list) #turn whole list to string
    
    display_dict_sorted[key] = key, ' '.join(sorted_list) #strip out everything and join with ' '.

display_dict_sorted

In [ ]:
submit_df = pd.DataFrame.from_dict(display_dict_sorted, orient='index')
submit_df.columns = ['display_id', 'ad_id']
submit_df.set_index(['display_id'], inplace=True)
# display(submit_df.head())
submit_df.to_csv('csv_submissions/prelim_model_v1.csv')


### Train on full train file

In [2]:
train = pd.read_csv("data/clicks_train.csv")

In [59]:
temp_tuple = train.groupby(['ad_id', 'clicked']).count().to_dict()


In [58]:
t1_dict{146773,1}

SyntaxError: invalid syntax (<ipython-input-58-892f4784a433>, line 1)

In [40]:
from timeit import default_timer as timer

unique_ads = train['ad_id'].unique().tolist()
ad_count_dict = {}
i = 0
ct = len(unique_ads)

start = timer()

for ad in unique_ads:
#     temp_tuple = () #3 elements for clicked, not clicked, and %
    
    temp = train.loc[train['ad_id'] == ad]
    
    appeared = temp.shape[0]
    
    not_clicked = temp.groupby('clicked').count().values[0,0]
    
    clicked = appeared - not_clicked
    
    percent_clicked = float(clicked)/appeared
    
    temp_tuple = (percent_clicked, appeared, clicked, not_clicked)
    
    
    ad_count_dict[ad] = temp_tuple
    
    if ((i % 1000) == 0):
        elapsed = (timer() - start) / 3600.0
        rate = elapsed / (i+1)
        print('Ad #{} of {}, Time left = {:,.2f} hours'.format(i+1, ct, (ct - i)*rate))
        
    i += 1
    


1970 1970 9807 9807
Ad #1 of 478950, Time left = 26.24 hours
199 199 1002 1002
18462 18462 38465 38465
1897 1897 56539 56539
109 109 226 226
4 4 30 30
202 202 1123 1123
0 0 1 1
50 50 525 525
316 316 564 564
1202 1202 13773 13773
916 916 2956 2956
3282 3282 16806 16806
145 145 554 554
153 153 1052 1052
312 312 7809 7809
723 723 4496 4496
6130 6130 22705 22705
426 426 5655 5655
187 187 1046 1046
236 236 1382 1382
111 111 409 409
202 202 496 496
282 282 1199 1199
9 9 87 87
10 10 110 110
5943 5943 72562 72562
2301 2301 18605 18605
6718 6718 78834 78834
3120 3120 8875 8875
8477 8477 194682 194682
11229 11229 33978 33978
789 789 11434 11434
380 380 2975 2975
0 0 1 1
329 329 10752 10752
713 713 3135 3135
6779 6779 23330 23330
18 18 393 393
2120 2120 10975 10975
4147 4147 40470 40470
1668 1668 16000 16000
31074 31074 150245 150245
46 46 152 152
2039 2039 5938 5938
5683 5683 16535 16535
2707 2707 19615 19615
10238 10238 53798 53798
8484 8484 19572 19572
2141 2141 11568 11568
3281 3281 14285 142

KeyboardInterrupt: 

In [ ]:
display_list = train['display_id'].unique().tolist()
display_dict = {}
for display in display_list:
    ad_list = train.loc[train['display_id'] == display]['ad_id'].tolist()
    
    display_dict[display] = ad_list

    
# step 3
    
display_dict_sorted = {}
for key, value in display_dict.iteritems():
    temp_list = []
    for e in value:
        temp_list.append(ad_count_dict[e][0])
    
    sorted_list = [x for (y,x) in sorted(zip(temp_list, value), reverse=True)]
    sorted_list = map(str, sorted_list) #turn whole list to string
    
    display_dict_sorted[key] = key, ' '.join(sorted_list) #strip out everything and join with ' '.


    
#step 4
    
submit_df = pd.DataFrame.from_dict(display_dict_sorted, orient='index')
submit_df.columns = ['display_id', 'ad_id']
submit_df.set_index(['display_id'], inplace=True)
submit_df.to_csv('csv_submissions/prelim_model_v1.csv')